In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import dill
import os
import random

import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

# Job.search()

## Visualization

In [12]:
job_counts = pd.read_feather('grouped/job_counts.feather')

start = pd.datetime(2017, 1, 1)
end = pd.datetime(2018, 7, 1)
duration = end - start
job_counts = job_counts[job_counts['posted_date'].isin(pd.date_range(start, end))]

print('{:0.3e} job posts over {} days across the US.'.format(job_counts['posts'].sum(), duration.days))

8.451e+07 job posts over 546 days across the US.


In [13]:
by_state = job_counts.groupby(['region', 'posted_date'])['posts'].sum()
by_state = by_state.reset_index()
states = by_state['region'].unique()

pop = pd.read_feather('other_data/census.feather')

## Jobs over time

In [3]:
per_day = job_counts.set_index('posted_date').groupby(pd.Grouper(freq='D'))['posts'].sum()
per_day = per_day.to_frame()
per_day.reset_index(inplace=True)

In [4]:
trace = go.Scatter(x=per_day['posted_date'], y=per_day['posts'])

layout = go.Layout(title='US Job Posts',
                   xaxis=dict(autorange=True, showgrid=True, zeroline=False, showline=True, showticklabels=True, title='Date'),
                   yaxis=dict(autorange=True, showgrid=True, zeroline=False, showline=True, showticklabels=True, title='Job Posts'))

data = [trace]

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='plots/total_posts_over_time.html', auto_open=False)
py.iplot(fig)

## By state

In [15]:
total = by_state.groupby(['posted_date']).sum()
by_state = by_state.set_index(['posted_date', 'region'])
by_state['date_ratio'] = by_state.div(total)
by_state.reset_index(level='posted_date', inplace=True)

In [16]:
layout = go.Layout(title='Job Posts by State',
                   xaxis=dict(autorange=True, showgrid=True, zeroline=False, showline=True, showticklabels=True, title='Date'),
                   yaxis=dict(autorange=True, showgrid=True, zeroline=True, showline=True, showticklabels=True, title='Job Posts'))

data = [go.Scatter(x=by_state.loc[state]['posted_date'], y=by_state.loc[state]['posts'],
                   name=state, stackgroup='none', hoverinfo='name+y') for state in states]

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='plots/state_posts_over_time.html', auto_open=False)
py.iplot(fig)

## Weekly distribution

In [5]:
def _add_day_of_week(df):
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
    df['day_of_week'] = pd.Categorical(df['posted_date'].dt.day_name(), categories=days, ordered=True)
    return None

In [6]:
_add_day_of_week(per_day)
week = per_day.groupby('day_of_week').sum()['posts']
total = week.sum()
week = week.to_frame()
week['ratio'] = week['posts']/total
week.sort_values('day_of_week', inplace=True)
week.reset_index(inplace=True)

In [7]:
trace = go.Bar(x=week['day_of_week'], y=week['ratio'])

data = [trace]

layout = go.Layout(title='Weekly Job Distribution',
                   xaxis=dict(autorange=True, showgrid=True, zeroline=False, showline=True, showticklabels=True, title='Day of Week'),
                   yaxis=dict(autorange=True, showgrid=True, zeroline=True, showline=True, showticklabels=True, title='Fraction of Job Posts'))

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='plots/weekly_distribution.html', auto_open=False)
py.iplot(fig)

## Jobs by location

In [17]:
tot_state = by_state.groupby('region')
tot_state = tot_state.sum()['posts'].to_frame()
tot_state['per_capita'] = tot_state['posts'].div(pop.set_index('region')['population'])

In [18]:
tot_state.sort_values('posts', ascending=False, inplace=True)

layout = go.Layout(title='Job Posts by State',
                   xaxis=dict(autorange=True, showgrid=True, zeroline=False, showline=True, showticklabels=True, title='State'),
                   yaxis=dict(autorange=True, showgrid=True, zeroline=True, showline=True, showticklabels=True, title='Job Posts'))

data = [go.Bar(x=tot_state.index, y=tot_state['posts'])]

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='plots/posts_by_state.html', auto_open=False)
py.iplot(fig)

In [13]:
tot_state.sort_values('per_capita', ascending=False, inplace=True)

layout = go.Layout(title='Job Posts by State per Capita',
                   xaxis=dict(autorange=True, showgrid=True, zeroline=False, showline=True, showticklabels=True, title='State'),
                   yaxis=dict(autorange=True, showgrid=True, zeroline=True, showline=True, showticklabels=True, title='Job Posts per Capita (arb)'))

data = [go.Bar(x=tot_state.index, y=tot_state['per_capita'])]

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='plots/posts_per_capita.html', auto_open=False)
py.iplot(fig)

## Jobs by location on map

In [19]:
by_state_week = []
for key, group in by_state.groupby('region'):
    if key != 'PR':
        temp = group.set_index('posted_date').groupby(pd.Grouper(freq='W'))['posts'].mean()
        temp = temp.to_frame().reset_index()
        temp['region'] = key
        by_state_week.append(temp)

by_state_week = pd.concat(by_state_week)
by_state_week = by_state_week.groupby(['posted_date', 'region']).sum()

by_state_week = pd.merge(by_state_week.reset_index(), pop.reset_index(drop=True), on=['region', 'region'])
by_state_week['per_capita'] = by_state_week['posts']/by_state_week['population']
by_state_week.set_index(['posted_date', 'region'], inplace=True)
by_state_week.sort_index(inplace=True)

In [20]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
most = by_state_week.max()['posts']
states = by_state_week.index.levels[1]
dates = by_state_week.index.levels[0]
data = []
for date in dates:
    data.append(go.Choropleth(locations=states, 
                              z=by_state_week.loc[date]['posts'], 
                              zmin=0,
                              zmax=most,
                              locationmode='USA-states',
                              colorscale='Reds', 
                              name=date.strftime('%Y-%m-%d'),
                              colorbar=dict(title='Job Posts',
                                            titleside='top'
                                           )
                             )
               )

# Create and add slider
steps = []
for i in range(len(data)):
    step = dict(method="restyle",
                args=["visible", [False] * len(data)],
                label=dates[i].strftime('%Y-%m-%d')
               )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(active=10, 
                currentvalue={"prefix": "Week of: "},
                steps=steps
               )
          ]
layout = go.Layout(geo_scope='usa', title='Job Posts Across US', sliders=sliders)

fig = go.Figure(data=data, layout=layout)
# # Make 10th trace visible
fig.data[10].visible = True

py.plot(fig, filename='plots/usa_total_posts.html', auto_open=False)
py.iplot(fig)

In [16]:
by_state_week = []
for key, group in by_state.groupby('region'):
    temp = group.set_index('posted_date').groupby(pd.Grouper(freq='W'))['posts'].sum()
    temp = temp.to_frame().reset_index()
    temp['region'] = key
    by_state_week.append(temp)

by_state_week = pd.concat(by_state_week)
by_state_week = by_state_week.groupby(['posted_date', 'region']).sum()

In [22]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
most = by_state_week.max()['per_capita']
states = by_state_week.index.levels[1]
dates = by_state_week.index.levels[0]
data = []
for date in dates:
    data.append(go.Choropleth(locations=states, 
                              z=by_state_week.loc[date]['per_capita'], 
                              zmin=0,
                              zmax=most,
                              locationmode='USA-states',
                              colorscale='Blues',
                              reversescale=True,
                              name=date.strftime('%Y-%m-%d'),
                              colorbar=dict(title='Job Posts Per Capita',
                                            titleside='top'
                                           )
                             )
               )

# Create and add slider
steps = []
for i in range(len(data)):
    step = dict(method="restyle",
                args=["visible", [False] * len(data)],
                label=dates[i].strftime('%Y-%m-%d')
               )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(active=10, 
                currentvalue={"prefix": "Week of: "},
                steps=steps
               )
          ]
layout = go.Layout(geo_scope='usa', title='Job Posts Across US', sliders=sliders)

fig = go.Figure(data=data, layout=layout)
# # Make 10th trace visible
fig.data[10].visible = True

py.plot(fig, filename='plots/usa_per_capita.html', auto_open=False)
py.iplot(fig)